In [1]:
#q1
!hdfs dfsadmin -D 'fs.defaultFS=hdfs://boss:9000' -report -live

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 7151009792 (6.66 GB)
DFS Remaining: 7150960640 (6.66 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.3:9866 (p4-dn-2.p4_default)
Hostname: 0355590adf2f
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 22228770816 (20.70 GB)
DFS Remaining: 3575480320 (3.33 GB)
DFS Used%: 0.00%
DFS Remaining%: 13.85%
Configured Cache Capacity: 0 (0 B)
Cache

In [23]:
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2023-11-26 21:01:52--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv.1’

hdma-wi-2021.csv.1  100%[===================>] 166.84M  22.8MB/s    in 8.0s    

2023-11-26 21:02:00 (20.9 MB/s) - ‘hdma-wi-2021.csv.1’ saved [174944099/174944099]



In [2]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [3]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [4]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/single.csv
!hdfs dfs -du -h hdfs://boss:9000/double.csv

166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [5]:
import requests

#q3
status = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
status.json()

{'FileStatus': {'accessTime': 1701036627785, 'blockSize': 1048576, 'childrenNum': 0, 'fileId': 16386, 'group': 'supergroup', 'length': 174944099, 'modificationTime': 1701036633888, 'owner': 'root', 'pathSuffix': '', 'permission': '644', 'replication': 1, 'storagePolicy': 0, 'type': 'FILE'}}


In [6]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()


{'Location': 'http://3c9b971ec4c3:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'}


In [7]:
block_size = status.json()['FileStatus']['blockSize']
length = status.json()['FileStatus']['length']
locs = {}
for i in range(0, length, block_size):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&noredirect=true&offset="+str(i))
    full_url = r.json()['Location']
    url = full_url.split(':')[1][2:]
    if url in locs:
        locs[url] += 1
    else:
        locs[url] = 1
#q5
locs


{'3c9b971ec4c3': 82, '0355590adf2f': 85}


In [40]:
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

In [50]:
f = hdfs.open_input_file("/single.csv")

#q6
f.read_at(10,0)

b'activity_y'


In [51]:
import io

In [52]:
txt = io.TextIOWrapper(f,newline='\n')
has_single = 0
txt.readline() #skip fields
line = txt.readline()
while line != "":
    if "Single Family" in line:
        has_single+=1
    line = txt.readline()

In [53]:
#q7
has_single

444874